# Digital Health Curation Analysis Scripts
## For any questions, please contact the corresponding author of the publication
==============================================================================================

### Files needed to be placed in the datapath folder specified below include:
- Grant portfolio, with curation fields added, downloaded from iSearch Grants Module: https://itools.od.nih.gov/isearch/grants/
    - filename labeled here as: dht_deduplicate2-iSearch_-_Grants-export_2023-03-28-11-27-03.xlsx
    - Please see the OPA website trainings to understand how to add curation fields to your iSearch portfolio: https://itools.od.nih.gov/help/curation

## Print-out statistics below are used in Supplemental Table 6 and reported in the text of the publication

In [1]:
datapath='C:\\Users\\radmantc\\Documents\\Digital Health Portfolio Analysis\\MasterData\\'#data path
outpath='C:\\Users\\radmantc\\\OneDrive - National Institutes of Health\\\Digital Health Portfolio Analysis\\'#data path
import pandas as pd
import numpy as np

In [2]:
#load iSearch curation portfolio results
dfi = pd.read_excel(datapath + 'dht_deduplicate2-iSearch_-_Grants-export_2023-03-28-11-27-03.xlsx')
dfi['Fiscal Year']=dfi['Fiscal Year'].astype(int)

C:\Users\radmantc\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
dfi.columns

Index(['index', 'Title', 'Abstract', 'Specific Aims',
       'Public Health Relevance', 'Condition', 'MeSH Extracted', 'Target',
       'Chemicals & Drugs', 'RCDC Categories',
       ...
       'Current Tools/Approaches (leonie.misquitta@nih.gov)',
       'FinishedCoding? (leonie.misquitta@nih.gov)',
       'requires2ndCuration? (leonie.misquitta@nih.gov)',
       'Notes (leonie.misquitta@nih.gov)',
       'v1 rationaleSelect (leonie.misquitta@nih.gov)', 'Added Date',
       'Record Index', 'curator', 'toCurate', 'toCurate2'],
      dtype='object', length=193)

In [7]:
#curation sensitivity:
#print(list(dfi.columns))

#knock out the non-curated files
dfc=dfi[~dfi['Relevant? (leonie.misquitta@nih.gov)'].isnull() | 
    ~dfi['Relevant? (christopher.hartshorn@nih.gov)'].isnull() | 
    ~dfi['Relevant? (audie.atienza@nih.gov)'].isnull() | 
    ~dfi['Relevant? (thomas.radman@nih.gov)'].isnull() | 
    ~dfi['Relevant? (josh.fessel@nih.gov)'].isnull() |
    ~dfi['Relevant? (pablo.cure@nih.gov)'].isnull()]

tmp=dfc[['Relevant? (leonie.misquitta@nih.gov)',
         'Relevant? (christopher.hartshorn@nih.gov)',
         'Relevant? (audie.atienza@nih.gov)',
         'Relevant? (thomas.radman@nih.gov)',
         'Relevant? (josh.fessel@nih.gov)',
         'Relevant? (pablo.cure@nih.gov)']]
tmpHigh=tmp=='highRelevance'
tmpLow=tmp=='poorRelevance'
dfc['highRelTot']=tmpHigh.sum(axis=1)
dfc['poorRelTot']=tmpLow.sum(axis=1)

#save DB:
import os
try:
    os.mkdir(outpath + '\\Outputs\\curations\\')
except OSError as error:
    print(error) 
dfc.to_csv(outpath + '\\Outputs\\curations\\' + 'Relevance.csv')

C:\Users\radmantc\AppData\Local\Temp\ipykernel_4320\1786434662.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['highRelTot']=tmpHigh.sum(axis=1)
C:\Users\radmantc\AppData\Local\Temp\ipykernel_4320\1786434662.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['poorRelTot']=tmpLow.sum(axis=1)


[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\radmantc\\\\OneDrive - National Institutes of Health\\\\Digital Health Portfolio Analysis\\\\Outputs\\curations\\'


In [8]:
#tied curations:
ties=dfc[dfc['highRelTot']==dfc['poorRelTot']]
nTies=ties.shape[0]
print(nTies)
ties.to_csv(outpath + '\\Outputs\\curations\\' + 'tiedRelevance.csv')

2


In [9]:
#knockout ties and count high vs. low rel counts:
tmp=dfc[dfc['highRelTot']!=dfc['poorRelTot']]
nCurated=tmp.shape[0]
nHigh=tmp[tmp['highRelTot']>tmp['poorRelTot']].shape[0]
print('Total Curated  = ', nCurated+nTies,
     '\nHigh Relevance = ',nHigh,
     '\nPoor Relevance  = ',nCurated-nHigh,
     '\nTies           = ',nTies)

Total Curated  =  482 
High Relevance =  421 
Poor Relevance  =  59 
Ties           =  2


In [10]:
dfi_join2=pd.merge(dfi,dfc, how='inner')  #dfi.join(dfc[['highRelTot','poorRelTot']])
dfi_join2.to_csv(outpath + '\\Outputs\\curations\\' + 'Subset_Relevance.csv')

In [11]:
#join the derived counts back to the dfi DB:
dfi_join=pd.merge(dfi,dfc, how='left')  #dfi.join(dfc[['highRelTot','poorRelTot']])

In [12]:
dfi_join[['highRelTot','poorRelTot']]

,highRelTot,poorRelTot
0,1.0,0.0
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
4468,NaN,NaN
4469,NaN,NaN
4470,NaN,NaN
4471,NaN,NaN


In [13]:
print(dfi.shape) #all grants
print(dfc.shape) #those from all grants w relevance
print(dfi_join.shape) #why does join have more?
print(dfi_join2.shape) #why does join have more?


(4473, 193)
(482, 195)
(4473, 195)
(482, 195)


In [14]:
### figure out the solutions for the categories.
#start by creating a column that is the concatenation of all curated values
tmp=dfi[[ 'Current Use/Focus (thomas.radman@nih.gov)',
         'Current Use/Focus (pablo.cure@nih.gov)',
        'Current Use/Focus (leonie.misquitta@nih.gov)',
        'Current Use/Focus (christopher.hartshorn@nih.gov)',
        'Current Use/Focus (audie.atienza@nih.gov)',
        'Current Use/Focus (josh.fessel@nih.gov)']]

dfc=tmp[tmp.notna().sum(axis=1)!=0]
dfc[dfc.isna()]=''
sep=' '
dfc['Current Use/Focus']=dfc['Current Use/Focus (thomas.radman@nih.gov)']\
+sep+dfc['Current Use/Focus (pablo.cure@nih.gov)']\
+sep+dfc['Current Use/Focus (leonie.misquitta@nih.gov)']\
+sep+dfc['Current Use/Focus (christopher.hartshorn@nih.gov)']\
+sep+dfc['Current Use/Focus (audie.atienza@nih.gov)']\
+sep+dfc['Current Use/Focus (josh.fessel@nih.gov)']
#print(list(tmp))
#print(dfc['Current Use/Focus'].unique())

dfc['isRemote']=dfc['Current Use/Focus'].str.contains('Remote Patient Monitoring/Clinical Decision Support')
dfc['isDataGath']=dfc['Current Use/Focus'].str.contains('Data Gathering/Database Development')
dfc['isIntervent']=dfc['Current Use/Focus'].str.contains('Interventional/Digital Rx')

dfc.to_csv(outpath + '\\Outputs\\curations\\' + 'useFocus.csv')

ntotal = dfc.shape[0]
nremote = dfc['isRemote'].sum()
ndatagath = dfc['isDataGath'].sum()
ninterv = dfc['isIntervent'].sum()
print( 'Total Curated = ', ntotal,
     '\nn (%) Remote Patient Monitoring/Clinical Decision Support = ',nremote,' (', round(nremote/ntotal*100,1), ')',
     '\nn (%) Data Gathering/Database Development = ',ndatagath,' (', round(ndatagath/ntotal*100,1), ')',
     '\nn (%) Interventional/Digital Rx = ',ninterv,' (', round(ninterv/ntotal*100,1), ')', 
     )

Total Curated =  332 
n (%) Remote Patient Monitoring/Clinical Decision Support =  107  ( 32.2 ) 
n (%) Data Gathering/Database Development =  194  ( 58.4 ) 
n (%) Interventional/Digital Rx =  131  ( 39.5 )


C:\Users\radmantc\AppData\Local\Temp\ipykernel_4320\256597059.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[dfc.isna()]=''
C:\Users\radmantc\AppData\Local\Temp\ipykernel_4320\256597059.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[dfc.isna()]=''
C:\Users\radmantc\AppData\Local\Temp\ipykernel_4320\256597059.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [15]:
#join the derived counts back to the dfi DB:
dfi_join=dfi_join.join(dfc[['isRemote','isDataGath','isIntervent']], how='left')  #dfi_join=dfi_join.join(dfc[['isRemote','isDataGath','isIntervent']])
print(dfi_join.shape)
# remove the duplicates
dfi_join.drop_duplicates(subset=['Appl ID'], keep='first', inplace=True, ignore_index=True)

(4473, 198)


In [16]:
dfi_join2=pd.merge(dfi,dfc, how='inner', left_index=True, right_index=True)  #dfi.join(dfc[['highRelTot','poorRelTot']])
dfi_join2.to_csv(outpath + '\\Outputs\\curations\\' + 'Subset_useFocus.csv')
print(dfi_join2.shape)

#dfi_join2

(332, 203)


In [17]:
print(dfi_join.shape, dfi_join['isRemote'].sum(), dfi_join['isDataGath'].sum(), dfi_join['isIntervent'].sum() )

(4473, 198) 107 194 131


In [18]:
#next category is current stage:
tmp=dfi[['Current Stage (thomas.radman@nih.gov)',
         'Current Stage (pablo.cure@nih.gov)',
        'Current Stage (leonie.misquitta@nih.gov)',
        'Current Stage (christopher.hartshorn@nih.gov)',
        'Current Stage (audie.atienza@nih.gov)',
        'Current Stage (josh.fessel@nih.gov)']]
dfc=tmp[tmp.notna().sum(axis=1)!=0]
#dfc[dfc.isna()]=''  #for some reason this line does not work for the stage variable???
dfc = dfc.where(dfc.notna(), other="")

sep=' '
dfc['Current Stage']=dfc['Current Stage (thomas.radman@nih.gov)']\
+sep+dfc['Current Stage (pablo.cure@nih.gov)']\
+sep+dfc['Current Stage (leonie.misquitta@nih.gov)']\
+sep+dfc['Current Stage (christopher.hartshorn@nih.gov)']\
+sep+dfc['Current Stage (audie.atienza@nih.gov)']\
+sep+dfc['Current Stage (josh.fessel@nih.gov)']
#print(list(tmp))
#print(dfc['Current Stage'].unique())

dfc['isClin']=dfc['Current Stage'].str.contains('Clinical/Regulatory Validation or Interventional')
dfc['isRnD']=dfc['Current Stage'].str.contains('Purely R and D')

dfc.to_csv(outpath + '\\Outputs\\curations\\' + 'stage.csv')

ntotal = dfc.shape[0]
nclin = dfc['isClin'].sum()
nRnD = dfc['isRnD'].sum()
print( 'Total Curated = ', ntotal,
     '\nn (%) Clinical/Regulatory Validation or Interventional = ',nclin,' (', round(nclin/ntotal*100,1), ')',
     '\nn (%) Purely R and D (e.g. prototype builds, device testing, V and V, early IDE) = ',nRnD,' (', round(nRnD/ntotal*100,1), ')', 
     )

Total Curated =  329 
n (%) Clinical/Regulatory Validation or Interventional =  135  ( 41.0 ) 
n (%) Purely R and D (e.g. prototype builds, device testing, V and V, early IDE) =  195  ( 59.3 )


In [19]:
#join the derived counts back to the dfi DB:
dfi_join=dfi_join.join(dfc[['isClin','isRnD']])
print(dfi_join.shape)

# remove the duplicates
dfi_join.drop_duplicates(subset=['Appl ID'], keep='first', inplace=True, ignore_index=True)

(4473, 200)


In [20]:
print(dfi_join.shape)
print(dfc['isClin'].sum(),  dfc['isRnD'].sum() )
print(dfi_join['isClin'].sum(),  dfi_join['isRnD'].sum() )

(4473, 200)
135 195
135 195


In [33]:
dfi_join2=pd.merge(dfi,dfc, how='inner', left_index=True, right_index=True)  #dfi.join(dfc[['highRelTot','poorRelTot']])
dfi_join2.to_csv(outpath + '\\Outputs\\curations\\' + 'Subset_stage.csv')


In [22]:
#next category is Tools:
tmp=dfi[['Current Tools/Approaches (thomas.radman@nih.gov)',
         'Current Tools/Approaches (pablo.cure@nih.gov)',
        'Current Tools/Approaches (leonie.misquitta@nih.gov)',
        'Current Tools/Approaches (christopher.hartshorn@nih.gov)',
        'Current Tools/Approaches (audie.atienza@nih.gov)',
        'Current Tools/Approaches (josh.fessel@nih.gov)']]
dfc=tmp[tmp.notna().sum(axis=1)!=0]
#dfc[dfc.isna()]=''  #for some reason this line does not work for the stage variable???
dfc = dfc.where(dfc.notna(), other="")

sep=' '
dfc['Current Tools/Approaches']=dfc['Current Tools/Approaches (thomas.radman@nih.gov)']\
+sep+dfc['Current Tools/Approaches (pablo.cure@nih.gov)']\
+sep+dfc['Current Tools/Approaches (leonie.misquitta@nih.gov)']\
+sep+dfc['Current Tools/Approaches (christopher.hartshorn@nih.gov)']\
+sep+dfc['Current Tools/Approaches (audie.atienza@nih.gov)']\
+sep+dfc['Current Tools/Approaches (josh.fessel@nih.gov)']
#print(list(tmp))
#print(dfc['Current Tools/Approaches'].unique())

dfc['isWear']=dfc['Current Tools/Approaches'].str.contains('Wearable')
dfc['isTele']=dfc['Current Tools/Approaches'].str.contains('Telemedicine/Telehealth')
dfc['isMobile']=dfc['Current Tools/Approaches'].str.contains('Mobile Apps')
dfc['isHome']=dfc['Current Tools/Approaches'].str.contains('In-home monitoring or other ambient measurement tools')

dfc.to_csv(outpath + '\\Outputs\\curations\\' + 'tools.csv')

ntotal = dfc.shape[0]
nwear = dfc['isWear'].sum()
ntele = dfc['isTele'].sum()
nmobile = dfc['isMobile'].sum()
nhome = dfc['isHome'].sum()
print( 'Total Curated = ', ntotal,
     '\nn (%) Wearable = ',nwear,' (', round(nwear/ntotal*100,1), ')',
     '\nn (%) Telemedicine/Telehealth = ',ntele,' (', round(ntele/ntotal*100,1), ')',
     '\nn (%) Mobile Apps = ',nmobile,' (', round(nmobile/ntotal*100,1), ')',
      '\nn (%) In-home monitoring or other ambient measurement tools = ',nhome,' (', round(nhome/ntotal*100,1), ')' 
     )


Total Curated =  327 
n (%) Wearable =  150  ( 45.9 ) 
n (%) Telemedicine/Telehealth =  88  ( 26.9 ) 
n (%) Mobile Apps =  158  ( 48.3 ) 
n (%) In-home monitoring or other ambient measurement tools =  62  ( 19.0 )


In [23]:
#join the derived counts back to the dfi DB:
dfi_join=dfi_join.join(dfc[['isWear','isTele','isMobile','isHome']])
# remove the duplicates
dfi_join.drop_duplicates(subset=['Appl ID'], keep='first', inplace=True, ignore_index=True)

In [24]:
print(dfc['isWear'].sum(), dfc['isTele'].sum(), dfc['isMobile'].sum(), dfc['isHome'].sum() )
print(dfi_join['isWear'].sum(), dfi_join['isTele'].sum(), dfi_join['isMobile'].sum(), dfi_join['isHome'].sum() )
dfi_join.shape

150 88 158 62
150 88 158 62


(4473, 204)

In [32]:
dfi_join2=pd.merge(dfi,dfc, how='inner', left_index=True, right_index=True)  #dfi.join(dfc[['highRelTot','poorRelTot']])
dfi_join2.to_csv(outpath + '\\Outputs\\curations\\' + 'Subset_tools.csv')


In [26]:
tmp=dfi_join[['Relevant? (leonie.misquitta@nih.gov)',
         'Relevant? (christopher.hartshorn@nih.gov)',
         'Relevant? (audie.atienza@nih.gov)',
         'Relevant? (thomas.radman@nih.gov)',
         'Relevant? (josh.fessel@nih.gov)',
         'Relevant? (pablo.cure@nih.gov)',
         'Current Use/Focus (thomas.radman@nih.gov)',
         'Current Use/Focus (pablo.cure@nih.gov)',
        'Current Use/Focus (leonie.misquitta@nih.gov)',
        'Current Use/Focus (christopher.hartshorn@nih.gov)',
        'Current Use/Focus (audie.atienza@nih.gov)',
        'Current Use/Focus (josh.fessel@nih.gov)',
        'Current Stage (thomas.radman@nih.gov)',
        'Current Stage (pablo.cure@nih.gov)',
        'Current Stage (leonie.misquitta@nih.gov)',
        'Current Stage (christopher.hartshorn@nih.gov)',
        'Current Stage (audie.atienza@nih.gov)',
        'Current Stage (josh.fessel@nih.gov)',
        'Current Tools/Approaches (thomas.radman@nih.gov)',
        'Current Tools/Approaches (pablo.cure@nih.gov)',
        'Current Tools/Approaches (leonie.misquitta@nih.gov)',
        'Current Tools/Approaches (christopher.hartshorn@nih.gov)',
        'Current Tools/Approaches (audie.atienza@nih.gov)',
        'Current Tools/Approaches (josh.fessel@nih.gov)']]
dfc=dfi_join[tmp.notna().sum(axis=1)!=0]
dfc.to_csv(outpath + '\\Outputs\\curations\\' + 'allCurationDerivations.csv')


In [27]:
dfc.shape

(482, 204)

In [29]:
tmp=dfi_join[[ 'highRelTot','poorRelTot',
              'isClin', 'isRnD',
              'isRemote','isDataGath','isIntervent',
              'isWear','isTele','isMobile','isHome'
         ]]
dfc2=dfi_join[tmp.notna().sum(axis=1)!=0]
dfc2.to_csv(outpath + '\\Outputs\\curations\\' + 'allCurationDerivations2.csv')
